In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/final_R2_final/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 8e-5
print(env.action_space)

/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Discrete(4)


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [6]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=False)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 10, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()
    a_env.close()

In [7]:
tf.reset_default_graph()
test_method()

/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from saves/breakout/final_R2_final/checkpoints/model2971808.cptk


/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


KeyboardInterrupt: 

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/julian/anaconda3/envs/gym/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/julian/anaconda3/envs/gym/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/julian/Desktop/Bachelor/Bachelor/ACER_ACTUALLY/Trainer.py", line 132, in test
    action = agent.get_action(state)
  File "/home/julian/Desktop/Bachelor/Bachelor/ACER_ACTUALLY/ACERNetwork.py", line 116, in get_action
    action, _ = self.get_action_with_probs(states)
  File "/home/julian/Desktop/Bachelor/Bachelor/ACER_ACTUALLY/ACERNetwork.py", line 111, in get_action_with_probs
    pi = self.get_pi(states)[0]
  File "/home/julian/Desktop/Bachelor/Bachelor/ACER_ACTUALLY/ACERNetwork.py", line 106, in get_pi
    pi = self.sess.run(self.pi, {self.tf_states: states})
  File "/home/julian/anaconda3/envs/gym/lib/python3.6/site-packages/tensorflow/python/client/session.py", 

NameError: name 'a_env' is not defined